In [ ]:
import numpy as np
import pandas as pd
import igraph as ig

In [ ]:
lass UnionFind:
    def __init__(self, n):
        self.parent = np.linspace(0, n-1, n, dtype=int)

    def find(self, i):
        if self.parent[i] == i: return i
        else: return self.find(self.parent[i])

    def link(self, i, j):
        assert self.find(i) != self.find(j) 
        self.parent[i] = j

    def union(self, i, j):
        if self.find(i) != self.find(j):
            self.link(self.find(i), self.find(j))


def kruskal(E, T, P):
    sorted = E.sort_values(by="weight", ascending=True)
    for index, edge in sorted.iterrows():
        if P.find(edge["u"]) != P.find(edge["v"]):
            T = T.append(edge, ignore_index=True)
            P.union(edge["u"], edge["v"])
    return E, T.sort_values(by=["u"]), P

kruskal_threshold = 10
def filter_kruskal(E, T, P):
    if len(E) < 20:
        E,T,P = kruskal(E, T, P)
    else:
        p = E.sample(1)["weight"].values[0] # Pick random weight as pivot element
        E_leq = E.loc[E["weight"] <= p]
        E_big = E.loc[E["weight"] > p]
        filter_kruskal(E_leq, T, P)
        E_big = filter(E_big, P)
        filter_kruskal(E_big, T, P)

    return E,T,P

def filter(E, P):
    T = pd.DataFrame()
    for index, edge in E.iterrows():
        if P.find(edge["u"]) != P.find(edge["v"]):
            T = T.append(edge, ignore_index=True)   
    return T 

In [ ]:
graph = pd.read_csv("../input_data/example.txt", sep=";", header=None)[1:].drop(columns=0)
graph = graph.rename(columns={1: "u", 2: "v", 3: "weight"})

mst = pd.DataFrame()
uf = UnionFind(len(graph))
E,T,P = filter_kruskal(graph, mst, uf)
T

In [ ]:
T["weight"].sum()